In [3]:
import numpy as np
import pandas as pd
from google.cloud import bigquery

client = bigquery.Client(project='hello-phase3')

/var/folders/57/2cl5z1650jqc2t11c02p4v7r0000gn/T/ipykernel_18487/1807553195.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


투자지표
- 수익성: '영업이익률', '영업이익＜당기＞'
- 성장성: '매출액증가율', '매출액＜전기＞', '매출액＜당기＞'
- 안정성: '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용'

In [4]:
sql_y = """
select stock_code, corp_name, year, account_name, amount
from `hello-phase3.naver_fn_data.investment_index_year`
where account_name in ('영업이익률', '영업이익＜당기＞', '매출액증가율', '매출액＜전기＞', '매출액＜당기＞', '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용')
"""

sql_q = """
select stock_code, corp_name, year, quarter, account_name, amount
from `hello-phase3.naver_fn_data.investment_index_quarter`
where account_name in ('영업이익률', '영업이익＜당기＞', '매출액증가율', '매출액＜전기＞', '매출액＜당기＞', '당좌비율', '당좌자산', '유동부채', '이자보상배율', '영업외이자비용')
"""

df_loaded_y = client.query_and_wait(sql_y).to_dataframe()
df_loaded_q = client.query_and_wait(sql_q).to_dataframe()

df_y = df_loaded_y.drop_duplicates(['stock_code', 'account_name', 'amount'], keep='first').sort_values(['year', 'account_name'])
df_q = df_loaded_q.drop_duplicates(['stock_code', 'account_name', 'amount'], keep='first').sort_values(['year', 'quarter', 'account_name'])

df_y['account_name'] = df_y['account_name'].str.replace('＜당기＞', '')
df_y['account_name'] = df_y['account_name'].str.replace('＜전기＞', '_전기')

df_q['account_name'] = df_q['account_name'].str.replace('＜당기＞', '')
df_q['account_name'] = df_q['account_name'].str.replace('＜전기＞', '_전기')

ValueError: Please install the 'db-dtypes' package to use this function.

안정성
1. 당좌비율 50% 미만
2. 이자보상배율 1 이상

In [ ]:
df_y.query('year == 2022').year.unique()

/var/folders/57/2cl5z1650jqc2t11c02p4v7r0000gn/T/ipykernel_12515/768349813.py:1: RuntimeWarning: Engine has switched to 'python' because numexpr does not support extension array dtypes. Please set your engine to python manually.
  df_y.query('year == 2022').year.unique()


<IntegerArray>
[2022]
Length: 1, dtype: Int64

In [ ]:
df_y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112779 entries, 142634 to 4182
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   stock_code    112779 non-null  object 
 1   corp_name     112779 non-null  object 
 2   year          112779 non-null  Int64  
 3   account_name  112779 non-null  object 
 4   amount        111345 non-null  float64
dtypes: Int64(1), float64(1), object(3)
memory usage: 5.3+ MB


In [2]:
(-5 - -10) / -10

-0.5

In [3]:
(-5 - 10) / 10

-1.5

In [4]:
(10 - -10) / -10

-2.0

In [20]:
import pandas as pd
data = [
    [10, 15],
    [10, 5],
    [10, 0],
    [10, -5],
    [-10, -15],
    [-10, -5],
    [-10, 0],
    [-10, 5],
    [-10, 10],
]
df = pd.DataFrame(data, columns=['prev', 'curr'])
df

,prev,curr
0,10,15
1,10,5
2,10,0
3,10,-5
4,-10,-15
5,-10,-5
6,-10,0
7,-10,5
8,-10,10


,prev,curr
0,NaN,5
1,NaN,-5
2,NaN,-10
3,NaN,-15
4,NaN,-5
5,NaN,5
6,NaN,10
7,NaN,15
8,NaN,20


In [9]:
df['sub'] = df['curr'] - df['prev']

In [10]:
df

,prev,curr,sub
0,10,15,5
1,10,5,-5
2,10,0,-10
3,10,-5,-15
4,-10,-15,-5
5,-10,-5,5
6,-10,0,10
7,-10,5,15
8,-10,10,20


In [14]:
df['pct_change'] = df[['prev', 'curr']].pct_change(axis=1)['curr']
df

,prev,curr,sub,pct_change
0,10,15,5,0.5
1,10,5,-5,-0.5
2,10,0,-10,-1.0
3,10,-5,-15,-1.5
4,-10,-15,-5,0.5
5,-10,-5,5,-0.5
6,-10,0,10,-1.0
7,-10,5,15,-1.5
8,-10,10,20,-2.0


In [15]:
df['direct_change'] = df['sub'] / df['prev']
df

,prev,curr,sub,pct_change,direct_change
0,10,15,5,0.5,0.5
1,10,5,-5,-0.5,-0.5
2,10,0,-10,-1.0,-1.0
3,10,-5,-15,-1.5,-1.5
4,-10,-15,-5,0.5,0.5
5,-10,-5,5,-0.5,-0.5
6,-10,0,10,-1.0,-1.0
7,-10,5,15,-1.5,-1.5
8,-10,10,20,-2.0,-2.0


In [18]:
import numpy as np

df['new_change'] = df['sub'] / np.abs(df['prev'])
df

,prev,curr,sub,pct_change,direct_change,new_change
0,10,15,5,0.5,0.5,0.5
1,10,5,-5,-0.5,-0.5,-0.5
2,10,0,-10,-1.0,-1.0,-1.0
3,10,-5,-15,-1.5,-1.5,-1.5
4,-10,-15,-5,0.5,0.5,-0.5
5,-10,-5,5,-0.5,-0.5,0.5
6,-10,0,10,-1.0,-1.0,1.0
7,-10,5,15,-1.5,-1.5,1.5
8,-10,10,20,-2.0,-2.0,2.0


In [19]:
df[['prev', 'curr', 'sub', 'new_change', 'pct_change']]

,prev,curr,sub,new_change,pct_change
0,10,15,5,0.5,0.5
1,10,5,-5,-0.5,-0.5
2,10,0,-10,-1.0,-1.0
3,10,-5,-15,-1.5,-1.5
4,-10,-15,-5,-0.5,0.5
5,-10,-5,5,0.5,-0.5
6,-10,0,10,1.0,-1.0
7,-10,5,15,1.5,-1.5
8,-10,10,20,2.0,-2.0


In [ ]:
[
    []
]